In [1]:
import numpy as np
import pickle

In [2]:
N_players = 5
players = [f'player{i}' for i in range(1, N_players+1)]

## Helper variables and functions

In [3]:
# === M ===
anti_dependence_transO = np.array([[0., 0.3, 0.7],
                                  [0.2, 0.6, 0.2],
                                  [0.7, 0.3, 0]])

low_transO = np.array([[0.5, 0.3, 0.2],
                       [0.25, 0.5, 0.25],
                       [0.2, 0.3, 0.5]])

avg_transO = np.array([[0.6, 0.3, 0.1],
                       [0.2, 0.6, 0.2],
                       [0.1, 0.3, 0.6]])

star_transO = np.array([[0.8, 0.2, 0],
                        [0.1, 0.8, 0.1],
                        [0, 0.2, 0.8]])

assymetric_up_transO = np.array([[0.7, 0.2, 0.1],
                        [0.1, 0.7, 0.2],
                        [0, 0.2, 0.8]])

assymetric_down_transO = np.array([[0.8, 0.2, 0],
                        [0.2, 0.7, 0.1],
                        [0.1, 0.2, 0.7]])

Ms = [avg_transO, star_transO,
      anti_dependence_transO, assymetric_up_transO, assymetric_down_transO]

# === N ===
anti_dependnece_transH = np.array([[0.1, 0.2, 0.7],
                                  [0.2, 0.6, 0.2],
                                  [0.7, 0.2, 0.1]])

low_transH = np.array([[0.7, 0.2, 0.1],
                       [0.2, 0.6, 0.2],
                       [0.1, 0.3, 0.7]])

avg_transH = np.array([[0.8, 0.2, 0.0],
                       [0.1, 0.8, 0.1],
                       [0.0, 0.2, 0.8]])

star_transH = np.array([[0.9, 0.1, 0.0],
                       [0.1, 0.8, 0.1],
                       [0.0, 0.1, 0.9]])

assymetric_up_transH = np.array([[0.7, 0.2, 0.1],
                        [0.1, 0.7, 0.2],
                        [0, 0.2, 0.8]])

assymetric_down_transH = np.array([[0.8, 0.2, 0],
                        [0.2, 0.7, 0.1],
                        [0.1, 0.2, 0.7]])

Ns = [avg_transH, star_transH, 
      anti_dependnece_transH, assymetric_up_transH, assymetric_down_transH]


# === emission_prob ===
low_emission = np.array([[0.5, 0.3, 0.2],
                       [0.25, 0.5, 0.25],
                       [0.2, 0.3, 0.5]])

avg_emission = np.array([[0.7, 0.2, 0.1],
                       [0.1, 0.8, 0.1],
                       [0.1, 0.2, 0.7]])

star_emission = np.array([[0.7, 0.3, 0.0],
                        [0.1, 0.8, 0.1],
                        [0.0, 0.1, 0.9]])

Es = [avg_emission, star_emission]

In [4]:
# ===  R ===
R_singleH = np.array([1] + [0] * len(players))
def R_singleHO(player):
    i = int(player[-1])
    arr = [0] * (len(players)+1)
    arr[0] = 0.8
    arr[i] = 0.2
    return np.array(arr)

def R_star(player, star):
    if player == star:
        return R_singleH
    arr = [0] * (len(players)+1)
    i = int(star[-1])
    arr[i] = 1
    return np.array(arr)

def R_star_plus_hidden(player, star):
    if player == star:
        return R_singleH
    arr = [0] * (len(players)+1)
    i = int(star[-1])
    arr[i] = 0.5
    arr[0] = 0.5
    return np.array(arr)

def R_two_stars(player, star1, star2):
    if player == star1 or player == star2:
        return R_singleH
    arr = [0] * (len(players)+1)
    i = int(star1[-1])
    j = int(star2[-1])
    arr[i] = 0.5
    arr[j] = 0.5
    return np.array(arr)

def R_two_stars_plus_hidden(player, star1, star2):
    if player == star1 or player == star2:
        return R_singleH
    arr = [0] * (len(players)+1)
    i = int(star1[-1])
    j = int(star2[-1])
    arr[i] = 0.25
    arr[j] = 0.25
    arr[0] = 0.5
    return np.array(arr)

def R_three_stars(player, star1, star2, star3):
    if player == star1 or player == star2 or player == star3:
        return R_singleH
    arr = [0] * (len(players)+1)
    i = int(star1[-1])
    j = int(star2[-1])
    k = int(star3[-1])
    arr[i] = 0.33
    arr[j] = 0.33
    arr[k] = 0.34
    return np.array(arr)

def R_three_stars_plus_hidden(player, star1, star2, star3):
    if player == star1 or player == star2 or player == star3:
        return R_singleH
    arr = [0] * (len(players)+1)
    i = int(star1[-1])
    j = int(star2[-1])
    k = int(star3[-1])
    arr[i] = 0.2
    arr[j] = 0.2
    arr[k] = 0.1
    arr[0] = 0.5
    return np.array(arr)

R_uniform = np.array([1/(1 + len(players))] * (len(players) + 1))

Rs = list()

In [5]:
def add_to_list(M, N, E, R, name):
    params = dict()
    params['M'] = M
    params['N'] = N
    params['E'] = E
    params['R'] = R
    params['name'] = name
    params_dict[name] = params
    Rs.append(R)

def add_to_dict(M, N, E, R, name):
    params = dict()
    params['M'] = M
    params['N'] = N
    params['E'] = E
    params['R'] = R
    params['name'] = name
    params_dict[name] = params

## Team profiles

In [6]:
params_dict = dict()

## Single Hidden

In [7]:
M = avg_transO
N = avg_transH
E = avg_emission
R = {player: R_singleH for player in players}
name = 'SingleH'
add_to_list(M, N, E, R, name)

## Single Hidden + Observed


In [8]:
M = avg_transO
N = avg_transH
E = avg_emission
R = {player: R_singleHO(player) for player in players}
name = 'SingleHO'
add_to_list(M, N, E, R, name)

## Single Star

In [9]:
M = star_transO
N = avg_transH
E = avg_emission
name = '1-Star'
for star in players:
    R = {player: R_star(player, star) for player in players}
    add_to_list(M, N, E, R, name)

## Single star + Hidden

In [10]:
M = star_transO
N = avg_transH
emission_prob = avg_emission
name = '1-Star + H'
for star in players:
    R = {player: R_star_plus_hidden(player, star) for player in players}
    add_to_list(M, N, E, R, name)

## Bad teammate - single star (BT1S)


In [11]:
M_bad_loser = np.array([[0.9, 0.1, 0],
                        [0.1, 0.8, 0.1],
                        [0.2, 0.3, 0.5]])
M = M_bad_loser
N = avg_transH
emission_prob = avg_emission
name = 'BT1S'

for star in players:
    R = {player: R_star(player, star) for player in players}
    add_to_list(M, N, E, R, name)

## Bad teammate - single star + Hidden (BT1S + H)

In [12]:
M_bad_loser = np.array([[0.9, 0.1, 0],
                        [0.1, 0.8, 0.1],
                        [0.2, 0.3, 0.5]])
M = M_bad_loser
N = avg_transH
emission_prob = avg_emission
name = 'BT1S + H'

for star in players:
    R = {player: R_star_plus_hidden(player, star) for player in players}
    add_to_list(M, N, E, R, name)

## Great teammate - Single star (GT1S)


In [13]:
M_great_teammate = np.array([[0.5, 0.3, 0.2],
                        [0.1, 0.8, 0.1],
                        [0.0, 0.1, 0.9]])
M = M_great_teammate
N = avg_transH
emission_prob = avg_emission
name = 'GT1S'

for star in players:
    R = {player: R_star(player, star) for player in players}
    add_to_list(M, N, E, R, name)

## Great teammate - single star + Hidden (GT1S + H)

In [14]:
M_great_teammate = np.array([[0.5, 0.3, 0.2],
                        [0.1, 0.8, 0.1],
                        [0.0, 0.1, 0.9]])
M = M_great_teammate
N = avg_transH
emission_prob = avg_emission
name = 'GT1S + H'

for star in players:
    R = {player: R_star_plus_hidden(player, star) for player in players}
    add_to_list(M, N, E, R, name)

## 2 stars


In [15]:
M = avg_transO
N = avg_transH
name = '2-Stars'

for star1 in players:
    for star2 in players:
        if star1 < star2:
            emission_prob = {p: avg_emission for p in players}
            emission_prob[star1] = star_emission
            emission_prob[star2] = star_emission
            R = {player: R_two_stars(player, star1, star2) for player in players}
            add_to_list(M, N, E, R, name)

## 2-stars with dependence (2-Stars + D)

In [16]:
M = avg_transO
N = avg_transH
name = '2-Stars + D'

for star1 in players:
    for star2 in players:
        if star1 < star2:
            emission_prob = {p: avg_emission for p in players}
            emission_prob[star1] = star_emission
            emission_prob[star2] = star_emission
            
            R = {player: R_two_stars(player, star1, star2) for player in players}
            R[star1] = R_star_plus_hidden(star1, star2) # Add star dependence
            R[star2] = R_star_plus_hidden(star2, star1)
            add_to_list(M, N, E, R, name)

## 2-stars + Hidden (2-Stars + H)

In [17]:
M = avg_transO
N = avg_transH
name = '2-Stars + H'

for star1 in players:
    for star2 in players:
        if star1 < star2:
            emission_prob = {p: avg_emission for p in players}
            emission_prob[star1] = star_emission
            emission_prob[star2] = star_emission
            R = {player: R_two_stars_plus_hidden(player, star1, star2) for player in players}
            add_to_list(M, N, E, R, name)

## 2-stars + Hidden with dependence

In [18]:
M = avg_transO
N = avg_transH
name = '2-Stars + HD'

for star1 in players:
    for star2 in players:
        if star1 < star2:
            emission_prob = {p: avg_emission for p in players}
            emission_prob[star1] = star_emission
            emission_prob[star2] = star_emission
            
            R = {player: R_two_stars_plus_hidden(player, star1, star2) for player in players}
            R[star1] = R_star_plus_hidden(star1, star2) # Add star dependence
            R[star2] = R_star_plus_hidden(star2, star1)
            add_to_list(M, N, E, R, name)

## 3-stars

In [19]:
M = avg_transO
N = avg_transH
name = '3-Stars'

for star1 in players:
    for star2 in players:
        for star3 in players:
            if star1 < star2 and star2 < star3:
                emission_prob = {player: avg_emission for player in players}
                emission_prob[star1] = star_emission
                emission_prob[star2] = star_emission
                emission_prob[star3] = star_emission

                R = {p: R_three_stars(p, star1, star2, star3) for p in players}
                add_to_list(M, N, E, R, name)

## 3-stars with dependence


In [20]:
M = avg_transO
N = avg_transH
name = '3-Stars + D'

for star1 in players:
    for star2 in players:
        for star3 in players:
            if star1 < star2 and star2 < star3:
                emission_prob = {player: avg_emission for player in players}
                emission_prob[star1] = star_emission
                emission_prob[star2] = star_emission
                emission_prob[star3] = star_emission

                R = {p: R_three_stars(p, star1, star2, star3) for p in players}
                R[star1] = R_two_stars_plus_hidden(star1, star2, star3)
                R[star2] = R_two_stars_plus_hidden(star2, star1, star3)
                R[star3] = R_two_stars_plus_hidden(star3, star1, star2)
                add_to_list(M, N, E, R, name)

## 3-stars + Hidden

In [21]:
M = avg_transO
N = avg_transH
name = '3-Star + H'

for star1 in players:
    for star2 in players:
        for star3 in players:
            if star1 < star2 and star2 < star3:
                emission_prob = {player: avg_emission for player in players}
                emission_prob[star1] = star_emission
                emission_prob[star2] = star_emission
                emission_prob[star3] = star_emission

                R = {p: R_three_stars_plus_hidden(p, star1, star2, star3) for p in players}
                add_to_list(M, N, E, R, name)

## 3-stars + Hidden with dependence

In [22]:
M = avg_transO
N = avg_transH
name = '3-Stars + HD'

for star1 in players:
    for star2 in players:
        for star3 in players:
            if star1 < star2 and star2 < star3:
                emission_prob = {player: avg_emission for player in players}
                emission_prob[star1] = star_emission
                emission_prob[star2] = star_emission
                emission_prob[star3] = star_emission

                R = {p: R_three_stars_plus_hidden(p, star1, star2, star3) for p in players}
                R[star1] = R_two_stars_plus_hidden(star1, star2, star3)
                R[star2] = R_two_stars_plus_hidden(star2, star1, star3)
                R[star3] = R_two_stars_plus_hidden(star3, star1, star2)
                add_to_list(M, N, E, R, name)

## Balanced

In [23]:
M = avg_transO
N = avg_transH
emission_prob = avg_emission
R = {player: R_uniform for player in players}
name = 'Balanced'
add_to_list(M, N, E, R, name)

## Generate new profiles
Combining all the M's, N's, E's, R's parameters used we create new profiles

In [24]:
i = 0
for M in Ms:
    for N in Ns:
        for E in Es:
            for R in Rs:
                add_to_dict(M, N, E, R, f'synthetic-{i}')
                i += 1

In [25]:
print(f'Total number of profiles = {len(params_dict)}')

Total number of profiles = 5667


# Save profiles to file

In [26]:
# Specify the file name
file_name = f'./profiles.pkl'

# Open the file in binary write mode
with open(file_name, 'wb') as file:
    pickle.dump(params_dict, file)

# Read profiles from file

In [27]:
with open(file_name, 'rb') as file:
    # Use pickle.load() to read the list from the file
    loaded_dict = pickle.load(file)

In [28]:
len(loaded_dict)

5667

In [29]:
Rs

[{'player1': array([1, 0, 0, 0, 0, 0]),
  'player2': array([1, 0, 0, 0, 0, 0]),
  'player3': array([1, 0, 0, 0, 0, 0]),
  'player4': array([1, 0, 0, 0, 0, 0]),
  'player5': array([1, 0, 0, 0, 0, 0])},
 {'player1': array([0.8, 0.2, 0. , 0. , 0. , 0. ]),
  'player2': array([0.8, 0. , 0.2, 0. , 0. , 0. ]),
  'player3': array([0.8, 0. , 0. , 0.2, 0. , 0. ]),
  'player4': array([0.8, 0. , 0. , 0. , 0.2, 0. ]),
  'player5': array([0.8, 0. , 0. , 0. , 0. , 0.2])},
 {'player1': array([1, 0, 0, 0, 0, 0]),
  'player2': array([0, 1, 0, 0, 0, 0]),
  'player3': array([0, 1, 0, 0, 0, 0]),
  'player4': array([0, 1, 0, 0, 0, 0]),
  'player5': array([0, 1, 0, 0, 0, 0])},
 {'player1': array([0, 0, 1, 0, 0, 0]),
  'player2': array([1, 0, 0, 0, 0, 0]),
  'player3': array([0, 0, 1, 0, 0, 0]),
  'player4': array([0, 0, 1, 0, 0, 0]),
  'player5': array([0, 0, 1, 0, 0, 0])},
 {'player1': array([0, 0, 0, 1, 0, 0]),
  'player2': array([0, 0, 0, 1, 0, 0]),
  'player3': array([1, 0, 0, 0, 0, 0]),
  'player4': arr

In [30]:
len(Rs)

113